In [5]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import numpy as np
import json
import datetime

import matplotlib.pyplot as plt


In [6]:
songs_clean_ts= pd.read_csv(os.path.join('data', 'songs_timeseries_2017_2021.csv'), index_col=0)
songs_clean_ts.head()

,song_id,num_day,total_countries,current_countries,inc,dec
0,4O4Z8VFczL8MxIOmqVWc1b,1,1,1,1,0
1,4O4Z8VFczL8MxIOmqVWc1b,2,2,1,1,1
2,4O4Z8VFczL8MxIOmqVWc1b,3,2,1,0,1
3,4O4Z8VFczL8MxIOmqVWc1b,4,2,1,0,1
4,4O4Z8VFczL8MxIOmqVWc1b,5,2,1,0,1


In [7]:
songs_to_keep = songs_clean_ts['song_id'].unique().tolist()

print(f'Number of songs under consideration: {len(songs_to_keep)}')

Number of songs under consideration: 4363


Dataset generated in notebook 01

In [8]:
ranking_df= pd.read_csv(os.path.join('data', 'clean_ranking_2017_2021.csv'), index_col=0, parse_dates=['Date', 'release_date'])
clean_ranking_df= ranking_df[ranking_df['song_id'].isin(songs_to_keep)]
clean_ranking_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date
2176,177,Footloose,Kenny Loggins,24993,4O4Z8VFczL8MxIOmqVWc1b,2018-01-11,Canada,4O4Z8VFczL8MxIOmqVWc1b,2018-01-11
3392,193,Tequila,Dan + Shay,23917,7Il2yWQmsqKgSkLQvHTPUI,2018-01-17,Canada,7Il2yWQmsqKgSkLQvHTPUI,2018-01-17
3594,195,Mine,Bazzi,23780,6tHWl8ows5JOZq9Yfaqn3M,2018-01-18,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15
3772,173,Mine,Bazzi,26500,6tHWl8ows5JOZq9Yfaqn3M,2018-01-19,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15
3978,179,Mine,Bazzi,24577,6tHWl8ows5JOZq9Yfaqn3M,2018-01-20,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15


In [9]:
clean_ranking_df = clean_ranking_df[clean_ranking_df['Region']!='World']

In [10]:
clean_ranking_df['Region'].unique()

array(['Canada', 'Switzerland', 'Chile', 'Columbia', 'CostaRica',
       'CzechRepublic', 'Germany', 'Denmark', 'DominicanRepublic',
       'Ecuador', 'Estonia', 'Spain', 'Finland', 'France',
       'UnitedKingdom', 'Greece', 'Guatemala', 'HongKong', 'Honduras',
       'Hungary', 'Indonesia', 'Ireland', 'Iceland', 'Italy', 'Japan',
       'Lithuania', 'Luxemborg', 'Latvia', 'Mexico', 'Malaysia',
       'Netherlands', 'Norway', 'NewZealand', 'Panama', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Paraguay', 'Sweden',
       'Singapore', 'Slovakia', 'ElSalvador', 'Turkey', 'Taiwan', 'USA',
       'Uruguay', 'Argentina', 'Austria', 'Australia', 'Belgium',
       'Bolivia', 'Brazil'], dtype=object)

In [11]:
clean_ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1359484 entries, 2176 to 34397
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   Position      1359484 non-null  int64         
 1   Track Name    1359484 non-null  object        
 2   Artist        1359484 non-null  object        
 3   Streams       1359484 non-null  int64         
 4   URL           1359484 non-null  object        
 5   Date          1359484 non-null  datetime64[ns]
 6   Region        1359484 non-null  object        
 7   song_id       1359484 non-null  object        
 8   release_date  1359484 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(5)
memory usage: 103.7+ MB


In [12]:
initial_date= datetime.datetime(2017, 1, 1)
def get_day_of_study_fn(d):
    return (d-initial_date).days 

In [13]:
clean_ranking_df['day_of_study'] = clean_ranking_df['Date'].apply(get_day_of_study_fn)
clean_ranking_df['release_month'] = clean_ranking_df['release_date'].dt.month

In [14]:
clean_ranking_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date,day_of_study,release_month
2176,177,Footloose,Kenny Loggins,24993,4O4Z8VFczL8MxIOmqVWc1b,2018-01-11,Canada,4O4Z8VFczL8MxIOmqVWc1b,2018-01-11,375,1
3392,193,Tequila,Dan + Shay,23917,7Il2yWQmsqKgSkLQvHTPUI,2018-01-17,Canada,7Il2yWQmsqKgSkLQvHTPUI,2018-01-17,381,1
3594,195,Mine,Bazzi,23780,6tHWl8ows5JOZq9Yfaqn3M,2018-01-18,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15,382,1
3772,173,Mine,Bazzi,26500,6tHWl8ows5JOZq9Yfaqn3M,2018-01-19,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15,383,1
3978,179,Mine,Bazzi,24577,6tHWl8ows5JOZq9Yfaqn3M,2018-01-20,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15,384,1


In [15]:
clean_ranking_df.to_csv(os.path.join('data','clean_ranking_2017_2021_v2.csv'))

In [16]:
clean_ranking_df.groupby(['song_id','Region'])['Date'].max()

song_id                 Region           
007ogFejDqJKzEXDUDF8Nf  Chile               2018-03-04
                        Paraguay            2018-06-11
00NpdPvXFERf1USRAcHkT3  CzechRepublic       2019-09-23
                        Slovakia            2019-12-05
00WvmRXTkPBZNhhRK3xfdy  Chile               2019-03-04
                                               ...    
7ztlf9mCrjoLXAYYf0LCYx  Malaysia            2019-03-27
                        Singapore           2019-03-15
                        Taiwan              2019-03-13
7zuM6FYdo5XhiocMPDkc0Q  Columbia            2020-08-20
                        DominicanRepublic   2020-08-14
Name: Date, Length: 19867, dtype: datetime64[ns]

In [17]:
song_max_dates_df =clean_ranking_df.groupby(['song_id','Region'])['Date'].max().to_frame()
song_max_dates_df.columns = ['max_date']
song_max_dates_df.head()

max_date
song_id                Region                  
007ogFejDqJKzEXDUDF8Nf Chile         2018-03-04
                       Paraguay      2018-06-11
00NpdPvXFERf1USRAcHkT3 CzechRepublic 2019-09-23
                       Slovakia      2019-12-05
00WvmRXTkPBZNhhRK3xfdy Chile         2019-03-04

In [18]:
song_dates_df =clean_ranking_df.groupby(['song_id','Region'])['Date'].min().to_frame()
song_dates_df.columns = ['min_date']
song_dates_df['max_date'] = song_max_dates_df['max_date']
song_dates_df['num_days_in_list'] = (song_dates_df['max_date']-song_dates_df['min_date']).dt.days

song_dates_df.head()

min_date   max_date  num_days_in_list
song_id                Region                                               
007ogFejDqJKzEXDUDF8Nf Chile         2018-02-26 2018-03-04                 6
                       Paraguay      2018-06-09 2018-06-11                 2
00NpdPvXFERf1USRAcHkT3 CzechRepublic 2019-09-01 2019-09-23                22
                       Slovakia      2019-08-31 2019-12-05                96
00WvmRXTkPBZNhhRK3xfdy Chile         2019-02-28 2019-03-04                 4

In [19]:
song_dates_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19867 entries, ('007ogFejDqJKzEXDUDF8Nf', 'Chile') to ('7zuM6FYdo5XhiocMPDkc0Q', 'DominicanRepublic')
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   min_date          19867 non-null  datetime64[ns]
 1   max_date          19867 non-null  datetime64[ns]
 2   num_days_in_list  19867 non-null  int64         
dtypes: datetime64[ns](2), int64(1)
memory usage: 558.4+ KB


In [20]:
song_dates_df['num_days_in_list'].describe()

count    19867.000000
mean       122.842352
std        203.336394
min          0.000000
25%          4.000000
50%         36.000000
75%        140.000000
max       1406.000000
Name: num_days_in_list, dtype: float64

In [21]:
song_dates_df['num_days_in_list'].plot.kde(grid=True);

In [22]:
song_dates_df['num_days_in_list'].value_counts(bins=20, normalize=True).plot(kind='bar', grid=True);

In [23]:
song_dates_df.to_csv(os.path.join('data', 'songs_dates_2017_2021.csv'))

In [24]:
song_dates_df['num_weeks_in_list'] = song_dates_df['num_days_in_list']/7


In [25]:
song_dates_df.describe()

,num_days_in_list,num_weeks_in_list
count,19867.000000,19867.000000
mean,122.842352,17.548907
std,203.336394,29.048056
min,0.000000,0.000000
25%,4.000000,0.571429
50%,36.000000,5.142857
75%,140.000000,20.000000
max,1406.000000,200.857143
